In [229]:
# Author : Parth Gera (gera2@illinois.edu)
# University of Illinois Urbana Champaign
# Data representation using folium and distance calculation between coordinates

## Importing Libraries ##

In [231]:
import requests
import math
import time
import pandas as pd
from datetime import datetime
import numpy as np
import os
import folium
from folium.plugins import MarkerCluster
import pandas as pd
from folium.plugins import TimestampedGeoJson
from datetime import datetime,date
import haversine as hs

## Extracting wildlife data for Champaign from inaturalist ## (A dataset of 5000 rows for now)

In [232]:
data = {
    'time_observed_at': list(),
    'species_guess': list(),
    'genus_name': list(),
    'rank': list(),
    'wikipedia_url': list(),
    'iconic_taxon_name': list(),
    'preferred_common_name': list(),
    'uri': list(),
    'longitude': list(),
    'latitude': list(),
    'place_guess': list()
}

#Fetching 5000 rows for now
for i in range(1,51):
    
    response = requests.get("https://api.inaturalist.org/v1/observations?place_id=1563&page={}&per_page=100".format(i))
    file_dict = response.json()
    if ('results' in file_dict):
        for j in range(100):

            if 'time_observed_at' in file_dict['results'][j]:
                data['time_observed_at'].append(file_dict['results'][j]['time_observed_at'])
            else:
                data['time_observed_at'].append(None)
            if 'species_guess' in file_dict['results'][j]:
                data['species_guess'].append(file_dict['results'][j]['species_guess'])
            else:
                data['species_guess'].append(None)

            if ('taxon' in file_dict['results'][j]) & (file_dict['results'][j]['taxon'] is not None) :
                #print(j)
                if 'name' in file_dict['results'][j]['taxon']:
                    data['genus_name'].append(file_dict['results'][j]['taxon']['name'])
                else:
                    data['genus_name'].append(None)
                if 'rank' in file_dict['results'][j]['taxon']:
                    data['rank'].append(file_dict['results'][j]['taxon']['rank'])
                else:
                    data['rank'].append(None)
                if 'wikipedia_url' in file_dict['results'][j]['taxon']:
                    data['wikipedia_url'].append(file_dict['results'][j]['taxon']['wikipedia_url'])
                else:
                    data['wikipedia_url'].append(None)
                if 'iconic_taxon_name' in file_dict['results'][j]['taxon']:
                    data['iconic_taxon_name'].append(file_dict['results'][j]['taxon']['iconic_taxon_name'])
                else:
                    data['iconic_taxon_name'].append(None)
                #print(j)
                if 'preferred_common_name' in file_dict['results'][j]['taxon']:
                    data['preferred_common_name'].append(file_dict['results'][j]['taxon']['preferred_common_name'])
                else:
                    data['preferred_common_name'].append(None)
            else:
                data['genus_name'].append(None)
                data['rank'].append(None)
                data['wikipedia_url'].append(None)
                data['iconic_taxon_name'].append(None)
                data['preferred_common_name'].append(None)
            if 'uri' in file_dict['results'][j]:
                data['uri'].append(file_dict['results'][j]['uri'])
            else:
                data['uri'].append(None)
            if 'geojson' in file_dict['results'][j]:
                data['longitude'].append(file_dict['results'][j]['geojson']['coordinates'][0])
                data['latitude'].append(file_dict['results'][j]['geojson']['coordinates'][1])
            else:
                data['longitude'].append(None)
                data['latitude'].append(None)
            if 'place_guess' in file_dict['results'][j]:
                data['place_guess'].append(file_dict['results'][j]['place_guess'])
            else:
                data['place_guess'].append(None)
    else:
        print(i)
        continue

In [233]:
#Converting to dataframe        
data_df = pd.DataFrame(data)
data_df.columns

Index(['time_observed_at', 'species_guess', 'genus_name', 'rank',
       'wikipedia_url', 'iconic_taxon_name', 'preferred_common_name', 'uri',
       'longitude', 'latitude', 'place_guess'],
      dtype='object')

## Using Folium for Plotting Data Collection Locations #

In [234]:
#Mapping colors basis iconic taxon name
id_dict={'Fungi': 'blue',
 'Aves': 'green',
 'Protozoa': 'red',
 'Insecta': 'cyan',
 'Plantae': 'magenta',
 'Mammalia': 'yellow',
 'Animalia': 'white',
 'Arachnida': 'gold',
 'Reptilia': 'lavendar',
 'Amphibia': 'tomato',
 'None': 'peru',
 'Mollusca': 'pink',
 'Actinopterygii': 'plum'}


data_df['intensity']=data_df['iconic_taxon_name'].map(id_dict)

In [235]:
#Removing NaT values and converting date to iso format
data_df['time_observed_at'].replace('NaT', np.nan, inplace = True)
data_df = data_df.dropna(axis=0, subset=['time_observed_at'])
data_df['time_observed_at']=pd.to_datetime(data_df['time_observed_at']).map(lambda x: x.isoformat())

In [236]:
#Creating data dictionary as an input for Folium Map
features = []

for lat,lan,intensity,time,genus_name,iconic_taxon_name in zip(data_df['latitude'],data_df['longitude'],data_df['intensity'],data_df['time_observed_at'],data_df['genus_name'],data_df['iconic_taxon_name']): 
    #time = str(time)
    feature = {
        'type': 'Feature',
        'geometry': {
            'type':'Point', 
            'coordinates':[lan,lat]
        },
        'properties': {
            'time': time,
            'style': {'color' : intensity},
            'icon': 'circle',
            'popup' : str(genus_name)+" ("+str(iconic_taxon_name)+")",
            'iconstyle':{
                'color': intensity,
                'fillOpacity': 0.8,
                'stroke': 'true',
                'radius': 7
            }
        }
    }
    features.append(feature)

In [237]:
#Centering the map on Champaign
champaign_coords = [40.1164, -88.2434]

#Creating the map
my_map = folium.Map(location = champaign_coords, zoom_start = 11)

#Plotting points on Folium using observation time as slider
TimestampedGeoJson(
        {'type': 'FeatureCollection',
        'features': features}
        , period='P1M'
        , add_last_point=True
        , auto_play=True
        , loop=False
        , max_speed=100
        , loop_button=True
        , date_options='YYYY/MM/DD'
        , time_slider_drag_update=True
    ).add_to(my_map)

#Displaying the map
my_map

##Move the sliders below:


## Distance calculation between 2 coordinates using haversine and cumulative distance of a location ##

In [238]:
#Creating a unique id column
data_df['id']=np.arange(len(data_df))
data_df['key']=1

#Creating another dataset to cross join
data_df_temp=data_df

#Cross join to map all points with all other points
result = pd.merge(data_df, data_df_temp, on ='key').drop("key", 1)   

In [239]:
#Function to calculate distance between 2 coordinates
def distance_function(loc1,loc2):
    
    dist=hs.haversine(loc1,loc2)
    return round(dist,2)

#Creating a distance column using the apply function
result['distance'] = result[["latitude_x","longitude_x","latitude_y","longitude_y"]].apply(lambda row:distance_function((row["latitude_x"],row["longitude_x"]),(row["latitude_y"],row["longitude_y"])),axis=1)


In [240]:
#Calculating cumulative distance of a point from all other points and merging with base data
distance_result=result.groupby(['id_x'])['distance'].sum()
distance_result_df=(pd.DataFrame(distance_result)).reset_index()
distance_result_df.rename(columns = {'id_x':'id'}, inplace = True)
data_df_final = pd.merge(data_df, distance_result_df, on ='id').drop("id", 1)

In [242]:
#Final data with Distance column added
data_df_final.head(100)

,time_observed_at,species_guess,genus_name,rank,wikipedia_url,iconic_taxon_name,preferred_common_name,uri,longitude,latitude,place_guess,intensity,key,distance
0,2021-06-17T08:31:25-05:00,None,Astragalus,genus,http://en.wikipedia.org/wiki/Astragalus,Plantae,milkvetches,https://www.inaturalist.org/observations/83471545,-88.396950,40.207808,"Mahomet, IL, US",magenta,1,94185.10
1,2021-06-17T08:34:15-05:00,None,Verbesina helianthoides,species,http://en.wikipedia.org/wiki/Verbesina_heliant...,Plantae,Yellow Crownbeard,https://www.inaturalist.org/observations/83471526,-88.397438,40.208580,"Mahomet, IL, US",magenta,1,94556.36
2,2021-06-17T08:51:38-05:00,butterfly milkweed,Asclepias tuberosa,species,http://en.wikipedia.org/wiki/Asclepias_tuberosa,Plantae,butterfly milkweed,https://www.inaturalist.org/observations/83471458,-88.398980,40.209308,"Mahomet, IL, US",magenta,1,95220.28
3,2021-06-17T08:51:51-05:00,None,Asclepias syriaca,species,http://en.wikipedia.org/wiki/Asclepias_syriaca,Plantae,common milkweed,https://www.inaturalist.org/observations/83471446,-88.398980,40.209317,"Mahomet, IL, US",magenta,1,95222.38
4,2021-06-17T09:27:51-05:00,None,Insecta,class,http://en.wikipedia.org/wiki/Insect,Insecta,Insects,https://www.inaturalist.org/observations/83471438,-88.399855,40.208483,"Mahomet, IL, US",cyan,1,95229.33
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,2021-06-16T11:36:52+00:00,Asian Lady Beetle,Harmonia axyridis,species,http://en.wikipedia.org/wiki/Harmonia_axyridis,Insecta,Asian Lady Beetle,https://www.inaturalist.org/observations/83278650,-88.217549,40.105203,"W Washington St, Urbana, IL, US",cyan,1,42831.76
96,2021-06-16T11:00:25+00:00,Kirtland's Snake,Clonophis kirtlandii,species,http://en.wikipedia.org/wiki/Kirtland's_snake,Reptilia,Kirtland's Snake,https://www.inaturalist.org/observations/83274763,-87.964833,40.243564,"Illinois, US",lavendar,1,139762.47
97,2021-05-22T08:32:14-04:00,None,Felis catus,species,http://en.wikipedia.org/wiki/Cat,Mammalia,Domestic Cat,https://www.inaturalist.org/observations/83273506,-88.250087,40.155071,"Luria Ln, Champaign, IL, US",yellow,1,52548.34
98,2021-06-16T10:47:06+00:00,Nosy Pill Woodlouse,Armadillidium nasatum,species,http://en.wikipedia.org/wiki/Armadillidium_nas...,Animalia,Nosy Pill Woodlouse,https://www.inaturalist.org/observations/83272608,-88.222825,40.090542,"University of Illinois, Urbana, IL, US",white,1,44226.34
